In [1]:
# In this game, we sell wine. We have N bottles of wine arranged on a shelf, each 
# having a certain initial price pi. We sell one bottle per year, either the one
# on the left end or the one on the right end. Over time, the price of wine goes up,
# so the total revenue from selling wine is given by:
#
#  P = sum_(y=1)^N y * p_y,
#
# where p_y is the initial price of the wine bottle sold in year y. How do we achieve
# maximal revenue?
#
# Note that the greedy solution is not optimal, as shown by the example:
#   p = [2, 3, 5, 1, 4]
# The greedy solution would be [1, 2, 5, 4, 3], but the optimal solution [1, 5, 4, 2, 3]
# takes a temporary 'loss' on selling the expensive bottle 5 earlier in order to get
# rid of the cheap bottle 4 earlier, for an overall win.
#
# This problem can be solved exactly by brute force recursion in O(2^N), exactly in O(N^2)
# by 'dynamic programming', or stochastically via Q-value reinforcement learning. This
# example thus illustrates the connection between DP and RL.
#

In [2]:
import numpy as np

# wine selling problem: brute force

In [3]:
#
# an optimal path and profit is returned,
# the path is not necessarily the unique
#
def max_profit(p):
    #
    # p = list of wine prices
    #

    # base case
    if len(p) == 1:
        return [], p[0]
    
    # recursion
    path_l, l = max_profit(p[1:]) 
    path_r, r = max_profit(p[:-1])
    path =  [['L', 'R'][l < r]] + [path_l, path_r][l < r]
    
    return path, sum(p) + max(l, r)

In [4]:
max_profit([2,3,5,1,4])

(['L', 'R', 'R', 'L'], 50)

# wine selling problem: memoization

In [5]:
#
# wine selling problem, with memoization
#
class DP():
    
    def __init__(self, p):
        #
        # p = list of wine prices
        #
        self.cache = np.zeros((len(p), len(p))) - 1
        self.p = p
        
    def max_profit(self, b, e):

        # base case
        if b == e:
            return self.p[b]
        
        # check if already computed
        if self.cache[b, e] < 0:
            self.cache[b, e] = sum(self.p[b:e+1]) + max(self.max_profit(b + 1, e), self.max_profit(b, e - 1))

        return self.cache[b, e]

In [7]:
dp = DP([2,3,5,1,4])
print dp.max_profit(0, len(dp.p)-1)

50.0


In [9]:
dp = DP([2,3,5,1,4,5,2,75,1,4,5,2,5,3,5,1,4,5,2,7,2,1,1,1,1,1,1,1,1])
dp.max_profit(0, len(dp.p)-1)

3484.0

# wine selling problem: Q-learning

In [27]:
#
# wine selling problem, solved by RL
#
class DP():
    
    def __init__(self, p):
        # p = list of wine prices
        np.random.seed(501)
        self.Q = np.random.randn(len(p), len(p), 2) # value of being in state (b, e, a)
        self.p = p

    def learn(self, eps=0.25, lr=0.5, gamma=1):

        b, e = 0, len(self.p) - 1
        
        # execute a roll out
        n = 0
        profit = 0
        path = []
        while b < e:
            n += 1

            if np.random.rand() < eps:
                # choose action at random
                a = int(np.random.rand() < 0.5)
            else:
                a = np.argmax(self.Q[b,e,:])
        
            rbase = n * self.p[[b, e][a > 0]]
            #print b, e, a, n, self.p[[b, e][a > 0]], rbase
            b1 = b + int(a == 0)
            e1 = e - int(a == 1)

            profit += rbase
            self.Q[b, e, a] = (1 - lr) * self.Q[b, e, a] + lr * (rbase + gamma * max(self.Q[b1, e1, :]))
            path.append(a)
            b = b1
            e = e1

        self.Q[b, e, :] = (n + 1)* self.p[b]
            
        return path, profit + (n + 1)* self.p[b]
    
    
    def play(self):
        return self.learn(0, 0)

In [28]:
x = DP([2,3,5,1,4,5,2,75,1,4,5,2,5,3,5,1,4,5,2,7,2,1,1,1,1,1,1,1,1])

In [29]:
[x.learn()[1] for _ in range(5000)]

/home/privitera/envs/rl-dev/lib/python2.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


[2495,
 2495,
 2498,
 2646,
 2498,
 2507,
 2629,
 2639,
 2495,
 2497,
 1985,
 1989,
 2647,
 2489,
 2498,
 2505,
 2491,
 2635,
 2238,
 2500,
 2497,
 2646,
 2248,
 2511,
 2640,
 2494,
 2499,
 2252,
 2342,
 2644,
 2510,
 2249,
 2511,
 2565,
 2156,
 2507,
 2493,
 2325,
 1996,
 2332,
 2501,
 2228,
 2497,
 2195,
 2243,
 2498,
 2255,
 2495,
 2256,
 2511,
 2502,
 2252,
 2497,
 2177,
 2256,
 2247,
 2467,
 2497,
 2629,
 2840,
 2498,
 2494,
 2484,
 2502,
 2495,
 2345,
 2492,
 2498,
 2504,
 2491,
 2500,
 2491,
 2488,
 2505,
 2507,
 2239,
 2484,
 2512,
 2502,
 2491,
 2263,
 2341,
 2500,
 2515,
 2264,
 2622,
 2264,
 2544,
 2629,
 2266,
 2625,
 2351,
 2420,
 2470,
 2479,
 2496,
 2650,
 2635,
 2543,
 2639,
 2570,
 2497,
 2560,
 2497,
 2512,
 2641,
 2631,
 2635,
 2320,
 2637,
 2609,
 2848,
 2178,
 2341,
 2250,
 2256,
 2557,
 2629,
 2269,
 2207,
 2633,
 2650,
 2566,
 2706,
 2491,
 2633,
 2338,
 2620,
 2243,
 2622,
 2697,
 2707,
 2437,
 2632,
 2647,
 2260,
 2634,
 2628,
 2639,
 2564,
 2566,
 2550,
 2631,

In [30]:
x.play()[1]

/home/privitera/envs/rl-dev/lib/python2.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


3484